# 1
고객群발견을 해보세요.\
1~5 번 변수에 대하여, K-means 클러스터링을 통해 고객 군집을 발견하세요.
이때 데이터 크기를 최소-최대 정규화를 적용하여 클러스터링하세요. 반복문을
사용하시어, K 가 2,4,6,8 일때의 Between Sum of Squares(모형의 inertia)를
비교해보세요. 그리고 k=8 에서의 각 군집의 특성을 파악해보세요\
ufb : 유저피드백\
usefulness : 리뷰의 유용성\
experience : 경험\
numofrv :리뷰수\
lenrv : 리뷰길이\
dup : 중복여부\
score : 점수


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
ofb = pd.read_csv("data_1_2.csv")

# 필요한 열 추출 및 정규화
X = ofb[['ufb', 'usefulness', 'experience', 'numofrv', 'lenrv']]
X_scaled = MinMaxScaler().fit_transform(X)

# 각 K 값에 대한 KMeans 모델을 피팅하고 inertia 출력
for K in [2,4,6,8]:
  kmeans=KMeans(n_clusters=K)
  kmeans.fit(X_scaled)
  kmeans.get_params()

  print(f'K={K}, Inertia: {kmeans.inertia_}')

In [ ]:
# K=8에서의 각 군집의 특성 파악
kmeans_k8 = KMeans(n_clusters=8)
ofb['cluster'] = kmeans_k8.fit_predict(X_scaled)
ofb

In [ ]:
# 각 군집의 특성 출력
cluster_features = ofb.groupby('cluster').mean() # ofb를 'cluster' 열을 기준으로 그룹화하고, 각 그룹에 대해 평균을 계산
print(cluster_features)

# 2
기존 제품의 판매 패턴을 mlxtend 모듈을 사용해서 발견해보세요.

최소 지지도는 0.01, 최소 리프트는2로 지정해보세요. 결과를 리프트가 3
이상이고, 컨피던스가 0.9 이상인 패턴을 발견해보세요.\
user : 유저\
artist : 아티스트\
qty : 판매된 제품 수

In [ ]:
!pip install mlxtend

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

In [ ]:
# 데이터 불러오기
gs = pd.read_csv("data_3.csv") #, delimiter=',', header=0
# 데이터 확인
print(gs.head())

In [ ]:
def func1( x ):
  if( x<=0.0):
    return False
  else:
    return True

In [ ]:
grouped = gs.groupby(['user', 'artist'])['qty'] #gs를 'user' 열과 'artist' 열을 기준으로 그룹화하고, 각 그룹에 대해 'qty' 열의 평균을 계산
# 두 컬럼을 기준으로 qty값을 정리/ 합계구하고, 형태 변환 / index새로 생성/NaN값은0변환 / user로 index 설정
transactions = grouped.sum().unstack().reset_index().fillna(0).set_index('user')
transactions1=transactions.applymap(func1)
transactions1

In [ ]:
#아래는 최소 지지도가 0.01이고 최소 리프트가 2인 조건으로 설정하여 연관 규칙을 찾고, 최소 리프트가 3 이상이고 컨피던스가 0.9 이상인 규칙을 선택하여 출력

frequent_itemsets = apriori(transactions1, min_support=0.01, use_colnames=True)
frequent_itemsets.sort_values(by='support', ascending=False) #내림차순

# 연관 규칙 생성
#(sort_values 함수를 사용하여 생성된 연관 규칙을 confidence, lift, support 순으로 내림차순으로 정렬)
#(가장 높은 confidence를 가진 규칙부터 차례로 확인 Confidence는 규칙의 신뢰도, Lift는 두 항목 간의 관련성, Support는 규칙이 데이터에서 얼마나 자주 발생하는지)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=2).sort_values(by = ['confidence', 'lift', 'support'], ascending=False)
rules

In [ ]:
#필요한 조건에 맞는 규칙 선택
filtered_rules = rules[(rules['lift'] >= 3) & (rules['confidence'] >= 0.9)]

# 결과 출력
print(filtered_rules)

#3.
판촉에 대한 고객의 피드백을 바탕으로, 고객 피드백을 통해 판촉에 대한 긍부정
반응을 분류하는 모형을 구축하려고 합니다.

data_5.csv에서는 고객의 피드백이 텍스트로 기록되어 있습니다.
텍스트마이닝을 거쳐, 중빈도(10~30)에 해당하는 단어들로 구성된 DTM이
data_5_2.csv 로 제공됩니다. 해당 파일의 첫 컬럼은 고객의 피드백이
긍정(1)인지 부정(0)인지 기록되어 있습니다. 이 데이터를 파티셔닝 후, 고객
피드백이 긍정인지를 분류하는 모형으로 적용해보세요.  

분류 모형 중 Decision Tree, Ensemble 모형 중 1개, 그리고 인공신경망을
사용하여 성능을 비교하고, 그 중 Ensemble 모형에는 GridSearch를 통해
최적의 파라미터를 찾아보세요.\
SMS : 쇼츠 메시지 서비스\
TV : 티비 광고\
News : 뉴스\
sales : 매출 수익

------
rating : 별점\
review : 리뷰

------
많은 단어들 : image.png ...

In [ ]:
# 필요한 라이브러리 불러오기
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text CountVectorizer #텍스트 데이터를 단어 빈도 행렬로 표현하여 자연어 처리 작업에서 특징 추출에 활용할
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV #그리드 서치 방법을 사용하여 모델의 하이퍼파라미터 튜닝을 수행
from sklearn.metrics import accuracy_score, classification_report #분류 모델의 성능 평가에 사용되며, 정확도 및 분류 보고서를 생성하는 데 활용

# 데이터 불러오기
data_4 = pd.read_csv('data_4.csv', sep='\t') #데이터가 탭으로 구분되어 있다면 sep='\t'를 사용하여 각 열을 올바르게 구분
data_5 = pd.read_csv('data_5.csv', encoding='cp949') #는 데이터 파일의 문자 인코딩 방식을 지정
data_5_2 = pd.read_csv('data_5_2.csv', index_col=0) #첫 번째 열을 인덱스로 사용하도록 설정

# 고객 피드백 데이터 전처리
X = data_5['review']
y = data_5['rating']

# DTM 데이터 추가 전에 index reset
data_5_2_reset = data_5_2.reset_index()

# DTM 데이터 추가
X_dtm = pd.concat([y, data_5_2_reset], axis=1)

# Train/Test 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(X_dtm.drop('rating', axis=1), X_dtm['rating'], test_size=0.2, random_state=42)

# Decision Tree 모델 훈련 및 평가
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)
dt_pred = dt_model.predict(X_test)
print("Decision Tree Accuracy:", accuracy_score(y_test, dt_pred))
print(classification_report(y_test, dt_pred))

# Random Forest 모델 훈련 및 평가
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
print("\nRandom Forest Accuracy:", accuracy_score(y_test, rf_pred))
print(classification_report(y_test, rf_pred))

# 인공신경망 모델 훈련 및 평가
mlp_model = MLPClassifier(random_state=42)
mlp_model.fit(X_train, y_train)
mlp_pred = mlp_model.predict(X_test)
print("\nNeural Network Accuracy:", accuracy_score(y_test, mlp_pred))
print(classification_report(y_test, mlp_pred))

# Random Forest에 GridSearchCV 적용
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)
best_rf_model = grid_search.best_estimator_

# 최적의 모델로 예측 및 평가
best_rf_pred = best_rf_model.predict(X_test)
print("\nBest Random Forest Accuracy:", accuracy_score(y_test, best_rf_pred))
print(classification_report(y_test, best_rf_pred))
print("Best Parameters:", grid_search.best_params_)